In [ ]:
import glob
import matplotlib.pyplot as plt
import pandas as pd
from numpy.fft import fft, ifft
import numpy as np
import scipy
import seaborn as sns

#IMPORTING FUNCTIONS FOR CLASSIFIERS
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedShuffleSplit, TimeSeriesSplit, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import mean_squared_error


#IMPORTING FUNCTIONS FROM ./defs

from defs import defs_

In [ ]:
#PATHS USED
path_quick_test = [
    "./database_raw/DatasetA/zzzAD1.txt",
    "./database_raw/DatasetA/zzzAD30.txt",
    "./database_raw/DatasetA/zzzAD12.txt",
]

path = glob.glob("./database_raw/DatasetA/zzzA*.txt", recursive=True)


In [ ]:
## Create pratical database with three different scenarios available on DATASETA, use to test code before apply on full database in order to save time
df_final = defs_.prepareDatabase(path);
### Instead of creating database every time, it was exported as a csv file named 'dataframesimples' on root of project, imported using following code
#######
#######
df_final.to_csv('dataframe_final')

df_final

In [ ]:
# Initiate DF's 

## Create full database with all data available on DATASETA
# df_final = defs_.prepareDatabase(path);
### Instead of creating database every time, it was exported as a csv file named 'dataframe_final' on root of project, imported using following code
#######
df_final = pd.read_csv('dataframe_final', dtype='unicode');
#######

## Create pratical database with three different scenarios available on DATASETA, use to test code before apply on full database in order to save time
# df_quick_test = defs_.prepareDatabase(path_quick_test);
### Instead of creating database every time, it was exported as a csv file named 'dataframesimples' on root of project, imported using following code
#######
df_quick_test = pd.read_csv('dataframe_simples', dtype='unicode');
#######

In [ ]:
# Start random forest Model

df_rf = df_quick_test

# Setting target variable
y = df_rf['Scenario']

# Setting other variables
X = df_rf.drop(columns=['Scenario'])
# X = X.drop(columns=['Sensor'])

# encoder = LabelBinarizer()
# X_lb = encoder.fit_transform(X['Sensor'])
# Sensor = X['Sensor']
X
X_new = pd.concat([X.loc[X['Sensor'].eq('S1').index],X.loc[X['Sensor'].eq('S2').index]],axis=1)
X_new = X_new.drop(columns=['Sensor'])
X_new

In [ ]:
le = LabelEncoder()
le.fit(X['Sensor'])
X['Sensor_encoded'] = le.transform(X['Sensor'])

X = X.drop(columns=['Sensor'])

X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=False) 

std_scaler = StandardScaler()
X_scaled_train = pd.DataFrame(std_scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_scaled_test = pd.DataFrame(std_scaler.transform(X_test),columns=X_train.columns,index=X_test.index)

X_scaled_train,X_scaled_test, y_train,y_test

In [ ]:
rf = RandomForestClassifier(n_estimators=100)

rf.fit(X_scaled_train,y_train)
y_pred = rf.predict(X_scaled_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)

rmse = np.sqrt(mse)

rmse

In [ ]:
et = ExtraTreesClassifier(n_estimators=100)

et.fit(X_scaled_train,y_train)
y_pred = rf.predict(X_scaled_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)

rmse = np.sqrt(mse)

rmse